# Homework 1

## Configuración inicial
    * Instalar Docker y desplegar un contenedor con PostgreSQL.

    * Crear una base de datos llamada supermercado_db.

    - Al inicializar el proyecto se crea automaticamente la base de datos mediante [`script`](./MOD2/LECTURE1/scripts/01_init_db.sql) con el siguiente comando 

    ```sql
    DROP DATABASE IF EXISTS market_db;
    CREATE DATABASE market_db;
    USE market_db;

# Homework 2

## Modelado de Datos
Crear las siguientes tablas con relaciones:

productos (id, nombre, categoría, precio, stock, sucursal_id)

sucursales (id, nombre, ubicación)

clientes (id, nombre, email, fecha_registro)

ventas (id, cliente_id, producto_id, fecha_venta, cantidad, total)

Definir claves primarias (PK), foráneas (FK) y restricciones (ON DELETE CASCADE/SET NULL).

   - Al igual que en el punto previo, la tablas se crean al inicializar el proyecto[`script`](./MOD2/LECTURE1/scripts/01_init_db.sql) con las siguientes instrucciones: 

    ```sql
    CREATE TABLE products (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    category VARCHAR(50) NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    stock INT NOT NULL,
    branch_id INT,
    FOREIGN KEY (branch_id) REFERENCES branchs(id) ON DELETE SET NULL
    );

    -- sucursales
    CREATE TABLE branchs (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        location VARCHAR(255) NOT NULL
    );

    -- clientes
    CREATE TABLE customers (
        id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100) NOT NULL,
        email VARCHAR(100) NOT NULL UNIQUE,
        registration_date DATE NOT NULL
    );
    -- tabla ventas
    CREATE TABLE sales (
        id INT AUTO_INCREMENT PRIMARY KEY,
        customer_id INT,
        product_id INT,
        sale_date DATE NOT NULL,
        quantity INT NOT NULL,
        total DECIMAL(10, 2) NOT NULL,
        FOREIGN KEY (customer_id) REFERENCES customers(id) ON DELETE CASCADE SET NULL,
        FOREIGN KEY (product_id) REFERENCES products(id) ON DELETE CASCADE SET NULL
    );
    ```

# Homework 3
## Operaciones CRUD
Insertar 10 registros en cada tabla.

Realizar consultas con JOIN para mostrar los productos vendidos por sucursal y por cliente.

Actualizar precios de productos por categoría.

Eliminar registros de ventas usando transacciones (BEGIN, COMMIT, ROLLBACK).

In [3]:
import utils.postgres_utils as postgres_utils

df = postgres_utils.run_query("SELECT * FROM sales LIMIT 10;")
df.head()


,id,customer_id,product_id,sale_date,quantity,total
0,3,3,3,2021-03-10,3,23.97
1,4,4,4,2023-04-20,1,12.99
2,6,6,6,2020-06-30,2,29.98
3,7,7,7,2020-06-30,1,11.49
4,8,8,8,2022-06-30,5,44.95


In [4]:
# inserrtar 10 registros en la tabla products
from data.fixtures import branches,products, customers, sales
from models.branches import Branch
from models.products import Product
from models.customers import Customer
from models.sales import Sale

import utils.postgres_utils as postgres_utils


# lista(zip(nombres, valores))
# Resultado: [('name', 'Sucursal 1'), ('location', 'Direccion 1')]

# diccionario = dict(Resultado)

# diccionario = {'name': 'Sucursal 1', 'location': 'Direccion 1'}

# branch_records = [dict(zip(["name", "location"], branch)) for branch in branches]
# product_records = [dict(zip(["name", "category", "price", "stock", "branch_id"], product)) for product in products]
Branch.bulk_insert(Branch.set_from_rows(branches))
Product.bulk_insert(Product.set_from_rows(products))
Customer.bulk_insert(Customer.set_from_rows(customers))
Sale.bulk_insert(Sale.set_from_rows(sales))


Branch bulk inserted successfully.
Product bulk inserted successfully.
Customer bulk inserted successfully.
Sale bulk inserted successfully.


* Realizar consultas con JOIN para mostrar los productos vendidos por sucursal y por cliente

In [5]:
import utils.postgres_utils as postgres_utils

query = """
SELECT customers.name AS name, products.name AS product , sale_date, quantity, total, branches.name AS branch
FROM sales s
JOIN customers ON s.customer_id = customers.id
JOIN products ON s.product_id = products.id
JOIN branches ON products.branch_id = branches.id
ORDER BY customers.name, sale_date DESC
"""

print("Sales Report:")
postgres_utils.run_query(query)



Sales Report:


,name,product,sale_date,quantity,total,branch
0,Ale,Producto J,2024-08-30,3,50.97,Sucursal 3
1,Ale,Producto J,2024-08-30,3,50.97,Sucursal 3
2,Axel,Producto F,2020-06-30,2,29.98,Sucursal 1
3,Axel,Producto F,2020-06-30,2,29.98,Sucursal 1
4,Carlos,Producto H,2022-06-30,5,44.95,Sucursal 2
5,Carlos,Producto H,2022-06-30,5,44.95,Sucursal 2
6,Gerardo,Producto H,2025-06-26,5,44.95,Sucursal 2
7,Gerardo,Producto H,2025-06-26,5,225.00,Sucursal 2
8,Gerardo,Producto H,2025-06-26,5,225.00,Sucursal 2
9,Gerardo,Producto B,2023-02-15,1,15.49,Sucursal 2


* Actualizar precios de productos por categoría.

In [6]:

from models.products import Product


Product.update_product_prices("Categoria 1", 10)

* Eliminar registros de ventas usando transacciones (BEGIN, COMMIT, ROLLBACK).

In [7]:
import utils.postgres_utils as pg_utils

query = """
SELECT * FROM sales
"""
print("Sales before deletion:")
pg_utils.run_query(query)


Sales before deletion:


,id,customer_id,product_id,sale_date,quantity,total
0,3,3,3,2021-03-10,3,23.97
1,4,4,4,2023-04-20,1,12.99
2,6,6,6,2020-06-30,2,29.98
3,7,7,7,2020-06-30,1,11.49
4,8,8,8,2022-06-30,5,44.95
5,9,9,9,2024-08-18,2,26.98
6,10,10,10,2024-08-30,3,50.97
7,11,1,8,2025-06-26,5,225.00
8,12,1,8,2025-06-26,5,225.00
9,13,1,8,2025-06-26,5,44.95


In [8]:
import utils.postgres_utils as pg_utils
from models.sales import Sale

Sale.delete_by_customer(1)

print("Sales after deletion:")
query = """
SELECT * FROM sales
"""
print("Sales before deletion:")
pg_utils.run_query(query)

Deleted sales for customer ID 1.
Sales after deletion:
Sales before deletion:


,id,customer_id,product_id,sale_date,quantity,total
0,3,3,3,2021-03-10,3,23.97
1,4,4,4,2023-04-20,1,12.99
2,6,6,6,2020-06-30,2,29.98
3,7,7,7,2020-06-30,1,11.49
4,8,8,8,2022-06-30,5,44.95
5,9,9,9,2024-08-18,2,26.98
6,10,10,10,2024-08-30,3,50.97
7,16,3,3,2021-03-10,3,23.97
8,17,4,4,2023-04-20,1,12.99
9,19,6,6,2020-06-30,2,29.98


# Homework 4

## Optimización
* Crear índices en columnas frecuentemente consultadas (ej: email en clientes, nombre en productos).

Usar EXPLAIN ANALYZE para comparar el rendimiento antes/después de los índices.

In [9]:
email_to_search = 'gera@mail.com'

query = f"""
    SELECT * FROM customers
    WHERE email ILIKE '%{email_to_search}%'
    """

print(f"Searching for customer with email: {email_to_search}")
postgres_utils.run_query(query=query)

Searching for customer with email: gera@mail.com


,id,name,email,registration_date
0,1,Gerardo,gera@mail.com,2023-01-01
1,11,Gerardo,gera@mail.com,2023-01-01


Resultado Query cliente por email antes del index 

![image](assets/img-query-email_before_index.png)

In [10]:
from models.customers import Customer
Customer.create_index(index_name='idx_customers_email',columns=['email'])

Index 'idx_customers_email' created successfully on 'customers'.


Resultado Query cliente por email luego del index . 
Se reduce nueve veces el tiempo de ejecución

![image.png](assets/query-plan.png)
<br/>
<br/>
<br/>

![image.png](assets/analyze.png)

# HW 5
## Automatización

* Crear un trigger que actualice el stock automáticamente al realizar una venta.


In [11]:
from models.sales_triggers import SalesTriggers
SalesTriggers.create_stock_update_trigger()

Error: trigger "trg_update_stock_after_sale" for relation "sales" already exists

[SalesTriggers] Trigger created successfully.


- Insert de nueva venta

In [12]:
from utils.postgres_utils import run_query

query ="""SELECT * FROM products"""

print("Select before sale to check stock")

run_query(query=query)


Select before sale to check stock


,id,name,category,price,stock,branch_id
0,12,Producto B,Categoria 2,15.49,50,2
1,14,Producto D,Categoria 3,12.99,75,3
2,15,Producto E,Categoria 2,9.49,120,2
3,17,Producto G,Categoria 3,11.49,60,3
4,18,Producto H,Categoria 2,8.99,90,2
5,20,Producto J,Categoria 3,16.99,40,3
6,2,Producto B,Categoria 2,15.49,49,2
7,4,Producto D,Categoria 3,12.99,74,3
8,5,Producto E,Categoria 2,9.49,116,2
9,7,Producto G,Categoria 3,11.49,59,3


In [13]:
from models.sales import Sale
# "customer_id", "product_id","sale_date", "quantity", "total"
Sale.insert(
    customer_id=1,
    product_id=8,
    sale_date="2025-06-26",
    quantity=5,
    total=225
)


Sale inserted successfully.


In [14]:
from utils.postgres_utils import run_query

query ="""SELECT * FROM products"""

print("Select after sale to check stock ")

run_query(query=query)


Select after sale to check stock 


,id,name,category,price,stock,branch_id
0,12,Producto B,Categoria 2,15.49,50,2
1,14,Producto D,Categoria 3,12.99,75,3
2,15,Producto E,Categoria 2,9.49,120,2
3,17,Producto G,Categoria 3,11.49,60,3
4,18,Producto H,Categoria 2,8.99,90,2
5,20,Producto J,Categoria 3,16.99,40,3
6,2,Producto B,Categoria 2,15.49,49,2
7,4,Producto D,Categoria 3,12.99,74,3
8,5,Producto E,Categoria 2,9.49,116,2
9,7,Producto G,Categoria 3,11.49,59,3


In [15]:

from models.sales_procedure import SalesProcedure
SalesProcedure.create_sale_procedure()


[SalesProcedure] Sale procedure created successfully.


In [ ]:
SalesProcedure.call_sale_procedure(
    customer_id=1,
    product_id=8,
    sale_date="2025-06-26",
    quantity=5,
)

# HW 6
## Documentación
* Exportar el esquema de la base de datos con pg_dump.

.[`schema`](./schema.sql)

* Incluir un archivo .sql con todas las consultas realizadas.

.[`queries realizadas`](./answers.sql)